In [1]:
import torch
from datasets import load_dataset, concatenate_datasets

c:\Users\lucas\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cnn = load_dataset("cnn_dailymail", "3.0.0", split="train")

c:\Users\lucas\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lucas\.cache\huggingface\hub\datasets--cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 11490/11490 [00:00<00:00, 34196.55 examples/s]


In [4]:
def format_example_for_gkd(example):

    user_message = {
        "role": "user",
        "content": f"Summarize the following article:\n\n{example['article']}"
    }

    assistant_message = {
        "role": "assistant",
        "content": example['highlights'] #EOS token needed?
    }

    return {"messages": [user_message, assistant_message]}


gkd_formatted_cnn = cnn.map(
    format_example_for_gkd,
    batched=False, 
    remove_columns=['article', 'highlights', 'id'],
    num_proc=4 # Use multiple cores if available
)

print(gkd_formatted_cnn)
print(gkd_formatted_cnn[0])

Map (num_proc=4): 100%|██████████| 287113/287113 [00:16<00:00, 17613.84 examples/s]

Dataset({
    features: ['messages'],
    num_rows: 287113
})
{'messages': [{'content': 'Summarize the following article:\n\nLONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or 

In [6]:
gkd_formatted_cnn.save_to_disk("data/gkd_formatted_cnn")

Saving the dataset (0/3 shards):   0%|          | 0/287113 [00:00<?, ? examples/s]

Saving the dataset (3/3 shards): 100%|██████████| 287113/287113 [00:04<00:00, 64472.27 examples/s]
